In [1]:
import json
import random
from collections import defaultdict
from itertools import chain
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold

Using TensorFlow backend.


In [2]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scoring = 'accuracy'

In [3]:
# Copy-pasted from the game.
# https://heroes.cdnvideo.ru/vk/v0326/locale/ru.json.gz

lib = {
  "0": "Карен",
  "1": "Аврора",
  "10": "Безликий",
  "11": "Чабба",
  "12": "Арахна",
  "13": "Орион",
  "14": "Фокс",
  "15": "Джинджер",
  "16": "Данте",
  "17": "Моджо",
  "18": "Судья",
  "19": "Темная Звезда",
  "2": "Галахад",
  "20": "Артемис",
  "21": "Маркус",
  "22": "Пеппи",
  "23": "Лиэн",
  "24": "Тесак",
  "25": "Исмаил",
  "26": "Лилит",
  "27": "Лютер",
  "28": "Цин Мао",
  "29": "Дориан",
  "3": "Кира",
  "30": "Корнелиус",
  "31": "Джет",
  "32": "Гелиос",
  "33": "Ларс",
  "34": "Криста",
  "35": "Йорген",
  "36": "Майя",
  "37": "Джу",
  "38": "Эльмир",
  "39": "Зири",
  "4": "Астарот",
  "40": "Небула",
  "41": "К'арх",
  "5": "Кай",
  "6": "Фобос",
  "7": "Тея",
  "8": "Сорвиголова",
  "9": "Хайди",
}

In [4]:
# Parse battle logs.

def parse_heroes(heroes: Iterable[dict], sign: int, result: dict):
    for hero in heroes:
        for hero_key in ('Level', 'Color', 'Star'):
            result[f'''{lib[str(hero['id'])]} {hero_key}'''] += sign * hero[hero_key.lower()]


def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    parse_heroes(battle.get('attackers') or battle['player'], +1, result)
    parse_heroes(battle.get('defenders') or battle['enemies'], -1, result)
        
    return {'Win': battle['win'], **result}

In [5]:
# Load the logs into a data frame.

def invert_column(series: pandas.Series):
    """
    Inverts the column to make an "opposite" battle.
    """
    return series == False if series.name == 'Win' else -series

lines = list(chain.from_iterable([
    open('battles.jsonl'),
    open('battles-lilia.jsonl'),
]))

numpy.random.seed(42)
battles = pandas.DataFrame([parse_battle(line) for line in lines]).fillna(value=0)
battles.drop_duplicates(inplace=True)  # because the files may contain duplicate battles
battles = pandas.concat((battles, battles.apply(invert_column)))  # add inverted battles
battles = battles.sample(frac=1)  # shuffle
battles.head()

,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
159,False,0.0,0.0,0.0,-8.0,-59.0,-3.0,0.0,0.0,0.0,...,0.0,-7.0,-55.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0
125,True,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
206,False,0.0,0.0,0.0,-9.0,-71.0,-3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173,False,-0.0,-0.0,-0.0,7.0,55.0,3.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
237,True,-0.0,-0.0,-0.0,9.0,75.0,3.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-7.0,-57.0,-3.0,-0.0,-0.0,-0.0


In [6]:
battles.describe()

,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,Астарот Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
count,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.00000,420.000000,...,420.000000,420.000000,420.000000,420.000000,420.000000,420.00000,420.000000,420.000000,420.000000,420.00000
mean,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
std,2.695351,21.138367,0.979504,3.698029,29.229403,1.858991,3.444064,28.489499,1.26642,5.120971,...,0.378415,1.245517,10.444078,0.378415,2.703308,21.89757,1.154011,0.871177,7.079164,0.38467
min,-9.000000,-70.000000,-4.000000,-9.000000,-75.000000,-6.000000,-8.000000,-69.000000,-3.00000,-10.000000,...,-2.000000,-8.000000,-71.000000,-3.000000,-8.000000,-70.00000,-3.000000,-7.000000,-60.000000,-3.00000
25%,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.00000,-6.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
max,9.000000,70.000000,4.000000,9.000000,75.000000,6.000000,8.000000,69.000000,3.00000,10.000000,...,2.000000,8.000000,71.000000,3.000000,8.000000,70.00000,3.000000,7.000000,60.000000,3.00000


In [7]:
x = battles.drop(['Win'], axis=1)
y = battles['Win']

In [8]:
classifier = LogisticRegression(max_iter=1000, fit_intercept=False)
param_grid = {'C': numpy.logspace(-6, 2, num=1000)}

numpy.random.seed(42)
grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring=scoring, n_jobs=4).fit(x, y)

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {grid_search.best_estimator_.classes_}')

scores = cross_val_score(grid_search.best_estimator_, x, y, scoring=scoring, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

Best score: 0.9666666666666667
Best params: {'C': 89.526571259964}
Classes: [False  True]
CV score: 0.9642857142857142 (std: 0.03875909665738025)


In [9]:
pandas.DataFrame({'Feature': x.columns, 'Importance': grid_search.best_estimator_.coef_[0]}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=20)

,Importance
Feature,
Криста Color,15.096416
Судья Star,9.707803
Астарот Star,9.520896
Аврора Star,9.306450
Орион Color,7.699611
Орион Star,7.121443
Моджо Color,7.099457
Моджо Star,6.982338
Зири Color,6.980713


In [10]:
result = pandas.concat((
    pandas.Series(grid_search.best_estimator_.predict(x), index=battles.index, name='Predicted'),
    pandas.Series(grid_search.best_estimator_.predict_proba(x)[:, 1], index=battles.index, name='Probability'),
    battles,
), axis=1)
result['Probability'] = result['Probability'].apply('{:.2f}'.format)
result.head()

,Predicted,Probability,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
159,False,0.03,False,0.0,0.0,0.0,-8.0,-59.0,-3.0,0.0,...,0.0,-7.0,-55.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0
125,True,0.97,True,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
206,False,0.00,False,0.0,0.0,0.0,-9.0,-71.0,-3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173,False,0.01,False,-0.0,-0.0,-0.0,7.0,55.0,3.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
237,True,1.00,True,-0.0,-0.0,-0.0,9.0,75.0,3.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-7.0,-57.0,-3.0,-0.0,-0.0,-0.0


In [11]:
# Experimental NN model.
# I want it to catch an interaction between different heroes (e.g. Йорген is good in conjunction with Исмаил).

def create_model() -> keras.Model:
    model = Sequential()
    model.add(Dense(1000, input_dim=x.columns.size, activation='sigmoid'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(1, input_dim=x.columns.size, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

keras_classifier = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)

numpy.random.seed(42)
scores = cross_val_score(keras_classifier, x, y, cv=cv)

print(f'Score: {scores.mean()} (std: {scores.std()})')

Score: 0.9690476176284608 (std: 0.021428571901624376)
